# Doc2Vec With GenSim

Or, creating fixed size vectors from arbitrary length paragraphs.

In [1]:
import nltk
import random
import numpy as np
from random import shuffle
from nltk.corpus import reuters
from nltk.tokenize import word_tokenize
from gensim.models import Doc2Vec
from gensim.models.doc2vec import TaggedDocument
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout
from keras.models import Sequential
from keras.models import load_model
from keras.optimizers import Adam
from sklearn.preprocessing.label import MultiLabelBinarizer

Using TensorFlow backend.


In [2]:
if not nltk.download('reuters', download_dir="./") or not nltk.download('punkt', download_dir="./"):
    print("Download Failed")

[nltk_data] Downloading package reuters to ./...
[nltk_data]   Package reuters is already up-to-date!
[nltk_data] Downloading package punkt to ./...
[nltk_data]   Package punkt is already up-to-date!


In [3]:
seed = 42
random.seed(seed)

## Exploring The Reuters Dataset

In [28]:
article_index = 1

# The ID for the article we are looking at.
article_id = reuters.fileids()[article_index]

# Get the article as a list of words (and punctuation)
article_text = reuters.raw(article_id)

# Get the tags associated with the article.
article_tags = reuters.categories(article_id)

print("ID:", article_id)
print("\nTags:", ", ".join(article_tags))
print("\nArticle:", article_text)

ID: test/14828

Tags: grain

Article: CHINA DAILY SAYS VERMIN EAT 7-12 PCT GRAIN STOCKS
  A survey of 19 provinces and seven cities
  showed vermin consume between seven and 12 pct of China's grain
  stocks, the China Daily said.
      It also said that each year 1.575 mln tonnes, or 25 pct, of
  China's fruit output are left to rot, and 2.1 mln tonnes, or up
  to 30 pct, of its vegetables. The paper blamed the waste on
  inadequate storage and bad preservation methods.
      It said the government had launched a national programme to
  reduce waste, calling for improved technology in storage and
  preservation, and greater production of additives. The paper
  gave no further details.
  




## Creating The Training Dataset For Gensim's Doc2Vec

In [5]:
tagged_documents = []

for i, file_id in enumerate(reuters.fileids()):
    document_words = reuters.raw(file_id)
    document_tags = reuters.categories(file_id)
    tagged_document = TaggedDocument(words=word_tokenize(document_words),
                                     tags=document_tags)
    tagged_documents.append(tagged_document)
    
shuffle(tagged_documents)

## Create The Doc2Vec Model

In [6]:
doc2vec_dimensions    = 300
ignore_word_count     = 2
amount_doc2vec_epochs = 100
amount_train_threads  = 12
doc2vec_location      = "./doc2vec_model.bin"

doc2vec = Doc2Vec(vector_size=doc2vec_dimensions, 
                  min_count=ignore_word_count, 
                  seed=seed,
                  workers=amount_train_threads)

In [7]:
doc2vec.build_vocab(tagged_documents)

In [8]:
doc2vec.train(tagged_documents, epochs=amount_doc2vec_epochs, total_examples=doc2vec.corpus_count)

In [9]:
doc2vec.save(doc2vec_location)

## Create The Training Dataset For The Neural Classifier

Because the neural network will need to output some sort of prediction to represent which tags are relevant for a given document, we need a vector representation that a loss function can be derived for the neural network for when it gets it wrong during supervised learning. Enter one-hot encodings.

In [10]:
one_hot_encoder = MultiLabelBinarizer()

all_category_combinations = [reuters.categories(file_id) for file_id in reuters.fileids()]
_ = one_hot_encoder.fit(all_category_combinations)

We will use the one hot encodings to create labelled data that the neural network will be able to use in a supervised learning setting. The data that is being labbeled will be represented as a GenSim Doc2Vec embedding. 

In [11]:
train_x = []
train_y = []
valid_x = []
valid_y = []

for file_id in reuters.fileids():
    
    # Get fixed size doc2vec embedding of the article.
    raw_article = reuters.raw(file_id)
    article_tokens = word_tokenize(raw_article)
    article_doc2vec_embedding = doc2vec.infer_vector(article_tokens)
    
    # Get onehot encoding of associated tags for the article.
    article_categories = reuters.categories(file_id)
    article_one_hot_label = one_hot_encoder.transform([article_categories])[0]
    
    # Append to the appropriate dataset.
    if file_id.startswith('training/'):
        train_x.append(article_doc2vec_embedding)
        train_y.append(article_one_hot_label)
    
    elif file_id.startswith('test/'):
        valid_x.append(article_doc2vec_embedding)
        valid_y.append(article_one_hot_label)
    
    else:
        raise ValueError("What id is this? {}".format(file_id))
        

In [12]:
def shuffle_arrays_in_unison(a, b):
    a = np.array(a)
    b = np.array(b)
    assert len(a) == len(b)
    permutation = np.random.permutation(len(a))
    return a[permutation], b[permutation]


train_x, train_y = shuffle_arrays_in_unison(train_x, train_y)
valid_x, valid_y = shuffle_arrays_in_unison(valid_x, valid_y)

print("train x shape: {}, train y shape: {}".format(train_x.shape, train_y.shape))
print("valid x shape: {}, valid y shape: {}".format(valid_x.shape, valid_y.shape))

train x shape: (7769, 300), train y shape: (7769, 90)
valid x shape: (3019, 300), valid y shape: (3019, 90)


## Exploring One Hot Encodings

Lets have a look at the first article in the dataset. We get the file id, then the tags associated with the article, and then convert the tags to a vector. Try changing the index to '2' to see multiple tags represented as a one-hot vector.

In [13]:
index = 0
file_id = reuters.fileids()[index]

# Get the tags from the article associated with the id.
file_tags = reuters.categories(file_id)

# Get the one hot encoding.
file_one_hot = one_hot_encoder.transform([file_tags])

# We can also get the tags back from the one hot encoding.
file_tags = one_hot_encoder.inverse_transform(file_one_hot)

amount_tag_categories = len(file_one_hot[0])

print("Categories:", file_tags[0])
print("\nOne Hot Encoding:\n", file_one_hot[0])
print("\nLength of One Hot:", amount_tag_categories)

Categories: ('trade',)

One Hot Encoding:
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0]

Length of One Hot: 90


## Create the Keras Classifier

In [14]:
def get_mlp(input_size, 
            hidden_sizes, 
            output_size, 
            amount_dropout=0.3, 
            activation='relu'):
    
    model = Sequential()
    
    # Add input to first hidden layer.
    model.add(Dense(input_dim=input_size, 
                    units=hidden_sizes[0],
                    activation=activation))
    model.add(Dropout(amount_dropout))
    
    # Loop over rest of hidden layers.
    for i in range(1, len(hidden_sizes) - 1):
        model.add(Dense(units=hidden_sizes[i],
                        activation=activation))
        model.add(Dropout(amount_dropout))
        
    # Add output layer.
    model.add(Dense(units=output_size, activation='sigmoid'))
    return model

In [15]:
learning_rate = 0.00005
batch_size = 32
amount_epochs = 400
hidden_sizes = [400, 400, 400, 200, 100]
classifier_model_location = "./classifier_model.bin"

In [16]:
# Build model.
model = get_mlp(input_size=doc2vec_dimensions,
                hidden_sizes=hidden_sizes, 
                output_size=amount_tag_categories)

# Compile model to computational graph.
model.compile(optimizer=Adam(lr=learning_rate), 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

# Saves the model with highest score after each training cycle
checkpointer = ModelCheckpoint(filepath=classifier_model_location, 
                               verbose=1, 
                               save_best_only=True)

Instructions for updating:
keep_dims is deprecated, use keepdims instead


## Training the neural network on the dataset

In [41]:
# Train the neural network
model.fit(x=train_x, 
          y=train_y, 
          validation_data=(train_x, train_y), 
          batch_size=batch_size, 
          epochs=amount_epochs, 
          callbacks=[checkpointer],
          shuffle=True)

Train on 7769 samples, validate on 7769 samples
Epoch 1/100
7769/7769 [==============================] - 2s - loss: 0.0110 - acc: 0.9963 - val_loss: 0.0071 - val_acc: 0.9978
Epoch 2/100
7769/7769 [==============================] - 2s - loss: 0.0109 - acc: 0.9963 - val_loss: 0.0068 - val_acc: 0.9979
Epoch 3/100
7769/7769 [==============================] - 1s - loss: 0.0108 - acc: 0.9963 - val_loss: 0.0067 - val_acc: 0.9979
Epoch 4/100
7769/7769 [==============================] - 2s - loss: 0.0108 - acc: 0.9963 - val_loss: 0.0067 - val_acc: 0.9979
Epoch 5/100
7769/7769 [==============================] - 2s - loss: 0.0105 - acc: 0.9964 - val_loss: 0.0066 - val_acc: 0.9979
Epoch 6/100
7769/7769 [==============================] - 2s - loss: 0.0104 - acc: 0.9965 - val_loss: 0.0066 - val_acc: 0.9980
Epoch 7/100
7769/7769 [==============================] - 2s - loss: 0.0103 - acc: 0.9965 - val_loss: 0.0064 - val_acc: 0.9980
Epoch 8/100
7769/7769 [==============================] - 2s - loss: 0.

7769/7769 [==============================] - 2s - loss: 0.0083 - acc: 0.9971 - val_loss: 0.0048 - val_acc: 0.9985
Epoch 29/100
7769/7769 [==============================] - 2s - loss: 0.0083 - acc: 0.9971 - val_loss: 0.0047 - val_acc: 0.9985
Epoch 30/100
7769/7769 [==============================] - 2s - loss: 0.0084 - acc: 0.9971 - val_loss: 0.0047 - val_acc: 0.9985
Epoch 31/100
7769/7769 [==============================] - 2s - loss: 0.0082 - acc: 0.9971 - val_loss: 0.0047 - val_acc: 0.9985
Epoch 32/100
7769/7769 [==============================] - 2s - loss: 0.0082 - acc: 0.9972 - val_loss: 0.0046 - val_acc: 0.9986
Epoch 33/100
7769/7769 [==============================] - 2s - loss: 0.0082 - acc: 0.9971 - val_loss: 0.0045 - val_acc: 0.9986
Epoch 34/100
7769/7769 [==============================] - 2s - loss: 0.0081 - acc: 0.9972 - val_loss: 0.0045 - val_acc: 0.9986
Epoch 35/100
7769/7769 [==============================] - 2s - loss: 0.0079 - acc: 0.9972 - val_loss: 0.0044 - val_acc: 0.99

7769/7769 [==============================] - 2s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0034 - val_acc: 0.9990
Epoch 57/100
7769/7769 [==============================] - 1s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0033 - val_acc: 0.9990
Epoch 58/100
7769/7769 [==============================] - 1s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0033 - val_acc: 0.9990
Epoch 59/100
7769/7769 [==============================] - 2s - loss: 0.0065 - acc: 0.9977 - val_loss: 0.0033 - val_acc: 0.9990
Epoch 60/100
7769/7769 [==============================] - 2s - loss: 0.0066 - acc: 0.9977 - val_loss: 0.0033 - val_acc: 0.9991
Epoch 61/100
7769/7769 [==============================] - 2s - loss: 0.0064 - acc: 0.9977 - val_loss: 0.0032 - val_acc: 0.9991
Epoch 62/100
7769/7769 [==============================] - 2s - loss: 0.0065 - acc: 0.9977 - val_loss: 0.0032 - val_acc: 0.9990
Epoch 63/100
7769/7769 [==============================] - 2s - loss: 0.0062 - acc: 0.9979 - val_loss: 0.0032 - val_acc: 0.99

7769/7769 [==============================] - 2s - loss: 0.0055 - acc: 0.9981 - val_loss: 0.0025 - val_acc: 0.9994
Epoch 85/100
7769/7769 [==============================] - 2s - loss: 0.0056 - acc: 0.9980 - val_loss: 0.0024 - val_acc: 0.9994
Epoch 86/100
7769/7769 [==============================] - 2s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.0023 - val_acc: 0.9994
Epoch 87/100
7769/7769 [==============================] - 2s - loss: 0.0053 - acc: 0.9981 - val_loss: 0.0023 - val_acc: 0.9994
Epoch 88/100
7769/7769 [==============================] - 2s - loss: 0.0051 - acc: 0.9982 - val_loss: 0.0023 - val_acc: 0.9994
Epoch 89/100
7769/7769 [==============================] - 2s - loss: 0.0050 - acc: 0.9982 - val_loss: 0.0022 - val_acc: 0.9994
Epoch 90/100
7769/7769 [==============================] - 2s - loss: 0.0051 - acc: 0.9982 - val_loss: 0.0023 - val_acc: 0.9994
Epoch 91/100
7769/7769 [==============================] - 2s - loss: 0.0051 - acc: 0.9982 - val_loss: 0.0022 - val_acc: 0.99

## Testing the neural network

In [42]:
model = load_model(classifier_model_location)
predictions = model.predict(valid_x)

# Binarise predictions
predictions[predictions < 0.5]  = 0
predictions[predictions >= 0.5] = 1

all_predicted_tags = one_hot_encoder.inverse_transform(predictions)
all_actual_tags = one_hot_encoder.inverse_transform(valid_y)

In [43]:
for predicted_tags, actual_tags in zip(all_predicted_tags, all_actual_tags):
    print('predicted: {} - actual: {}'.format(predicted_tags, actual_tags))

predicted: ('reserves',) - actual: ('reserves',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('crude',) - actual: ('crude',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('gold',) - actual: ('gold',)
predicted: ('interest',) - actual: ('interest',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('money-fx',) - actual: ('dlr', 'money-fx', 'yen')
predicted: ('crude',) - actual: ('crude', 'gas')
predicted: ('earn',) - actual: ('earn',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('money-fx',) - actual: ('money-fx',)
predicted: ('dlr', 'money-fx', 'yen') - actual: ('dlr', 'money-fx', 'yen')
predicted: ('grain',) - actual: ('grain', 'ship')
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('iron-steel',) - actual: ('iron-steel',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: () - actual: ('earn',)
p

predicted: ('earn',) - actual: ('earn',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('corn', 'grain') - actual: ('corn', 'grain')
predicted: ('earn',) - actual: ('earn',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('gold',) - actual: ('gold', 'platinum')
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('acq',) - actual: ('acq',)
predicted: ('crude',) - actual: ('propane',)
predicted: ('coffee',) - actual: ('coffee',)
predicted: ('veg-oil',) - actual: ('veg-oil',)
predicted: ('sugar',) - actual: ('sugar',)
predicted: ('earn',) - actual: ('earn',)
predicted: ('earn',) - actual: ('earn',)
predicted: () - actual: ('carcass',)
predicted: ('earn',) - act

In [91]:
fake_article = """
GOOGLE TO BUY FACEBOOK"
(Alphabet Inc) said its (Google Inc) subsidiary agreed to buy privately-owned Facebook for an undisclosed amount of cash."""
fake_tokens = word_tokenize(fake_article)
fake_embedding = doc2vec.infer_vector(fake_tokens)
fake_input = fake_embedding.reshape((-1, doc2vec_dimensions))

fake_prediction = model.predict(fake_input)
fake_prediction[fake_prediction < 0.5]  = 0
fake_prediction[fake_prediction >= 0.5] = 1
fake_tags = one_hot_encoder.inverse_transform(fake_prediction)
fake_tags

[('acq',)]